# Import des données.

La base de données est fabriqué à partir d'un dataset au préalable traité via une reconstruction de données effectué dans le module 3.

Ce dataset se trouve dans le fichier CSV `data/00-reconstructed-dataset.csv'

**NOTA : Ce code étant du code jetable peu de soin seront fait sur le controle d'erreur.**

In [1]:
from select import select

# Bootstrap
import pandas as pd

In [2]:
from sqlalchemy import create_engine, select, func, delete, update
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey

## Construction du model

### Recupération de la structure du fichier

In [3]:
df = pd.read_csv('data/00-reconstructed-dataset.csv')
# Conversion de la date_creation_compte en vrai date
df['date_creation_compte'] = pd.to_datetime(df['date_creation_compte']).dt.date
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   nom                    4707 non-null   object 
 1   prenom                 4707 non-null   object 
 2   age                    4707 non-null   int64  
 3   taille                 4707 non-null   float64
 4   poids                  4707 non-null   float64
 5   sexe                   4707 non-null   object 
 6   sport_licence          4707 non-null   object 
 7   niveau_etude           4707 non-null   object 
 8   region                 4707 non-null   object 
 9   smoker                 4707 non-null   object 
 10  nationalite_francaise  4707 non-null   object 
 11  revenu_estime_mois     4707 non-null   float64
 12  situation_familiale    4707 non-null   object 
 13  historique_credits     4707 non-null   float64
 14  risque_personnel       4707 non-null   float64
 15  date

## Creation de la base from scratch

In [4]:
DATABASE_FILE = 'data/database.db'
DATABASE_URI = f'sqlite:///./{DATABASE_FILE}'

Base = declarative_base()

# Suppression de la base





In [5]:
# Models
class Raw_data(Base):
    __tablename__ = "raw_data"

    # Clé primaire auto-incrémentée (bonne pratique)
    id = Column(Integer, primary_key=True, index=True)

    # Colonnes mappées depuis le DataFrame
    nom = Column(String)
    prenom = Column(String)
    age = Column(Integer)
    taille = Column(Float)
    poids = Column(Float)
    sexe = Column(String)
    sport_licence = Column(String)
    niveau_etude = Column(String)
    region = Column(String)
    smoker = Column(String) # voir a mettre un bool car oui/non
    nationalite_francaise = Column(String) # voir a mettre un bool car oui/non
    revenu_estime_mois = Column(Float)
    situation_familiale = Column(String)
    historique_credits = Column(Float)
    risque_personnel = Column(Float)
    date_creation_compte = Column(Date)
    score_credit = Column(Float)
    loyer_mensuel = Column(Float)
    montant_pret = Column(Float)


## Création des models spécialisé

In [6]:
# Model Pret
# Ca correspond à chaque ligne du dataset, dans le dataset, ca correspond à un contrat par ligne

class Pret(Base):
    __tablename__ = "pret"

    # Clé primaire auto-incrémentée (numero de pret)
    num_pret = Column(Integer, primary_key=True, index=True)

    age = Column(Integer)
    taille = Column(Float)
    poids = Column(Float)
    sport_licence = Column(String)
    region = Column(String)
    smoker = Column(String) # voir a mettre un bool car oui/non
    revenu_estime_mois = Column(Float)
    situation_familiale = Column(String)
    historique_credits = Column(Float)
    risque_personnel = Column(Float)
    date_creation_compte = Column(Date)
    score_credit = Column(Float)
    loyer_mensuel = Column(Float)
    montant_pret = Column(Float)

    # Clé étrangère
    # Client
    client_id = Column(Integer, ForeignKey('client.id'))
    client = relationship("Client", back_populates="prets")

    # Region
    region_id = Column(Integer, ForeignKey('region.id'))
    region = relationship("Region", back_populates="prets")

    # Niveau d'étude
    niveau_etude_id = Column(Integer, ForeignKey('niveau_etude.id'))
    niveau_etude = relationship("Niveau_etude", back_populates="prets")


In [7]:
# Model client
class Client(Base):
    __tablename__ = "client"

    # Clé primaire auto-incrémentée (bonne pratique)
    id = Column(Integer, primary_key=True, index=True)

    # Colonnes mappées depuis le DataFrame
    nom = Column(String)
    prenom = Column(String)
    sexe = Column(String)
    nationalite_francaise = Column(String) # voir a mettre un bool car oui/non

    # Relations
    prets = relationship("Pret", back_populates="client")

    def __repr__(self):
        """Méthode qui renvoie une représentation sous forme de chaîne de caractères du client.

        La chaîne contient l'identifiant, le nom et le prénom du client.

        Returns :
            str : Une chaîne formatée avec l'ID, le nom et le prénom du client.
        """
        return f"<Client N°{self.id} ; {self.nom} {self.prenom}>"


In [8]:
# Dictionnaires
class Region(Base):
    __tablename__ = "region"

    id = Column(Integer, primary_key=True, index=True)
    nom = Column(String)

    # Relations
    prets = relationship("Pret", back_populates="region")

class Niveau_etude(Base):
    __tablename__ = "niveau_etude"

    id = Column(Integer, primary_key=True, index=True)
    nom = Column(String)

    # Relations
    prets = relationship("Pret", back_populates="niveau_etude")


## Création de la base

In [9]:
# Creation de la base vide
engine = create_engine(DATABASE_URI)

# A voir si ca recrer les tables ou si ca zap ce qui existe deja.
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()


## Populate de la base

**/!\\ Les cellules on un flag `populate_*` pour executer ou non les opération de remplissage des tables. Ca permet de relancer le notebook depuis le debut sans recréer les données déjà dans la base /!\\**

In [10]:
populate_raw = False
populate_client = False
populate_dico = False
populate_pret = False


In [11]:
# Remplissage brute avec les données du dataset
if populate_raw:
    for line in df.itertuples(index=False):
        dictline = line._asdict()
        new_raw_data = Raw_data(**dictline)
        session.add(new_raw_data)
        session.commit()




### Analyse des clients

On va remplir la base des client en les regroupant par nom, prenom, sexe et nationalite_française, c'est un choix arbitraire pour créer des clients multi possesseur de prêt.

In [12]:
# Construction de la requete

stm_client_groupe_t = select(
        # Raw_data,
        Raw_data.nom,
        Raw_data.prenom,
        Raw_data.nationalite_francaise,
        Raw_data.sexe
    ).group_by(  # GROUP BY ...
        Raw_data.nom,
        Raw_data.prenom,
        Raw_data.nationalite_francaise,
        Raw_data.sexe
    )

print(stm_client_groupe_t)
# TODO : Déterminé s'il est possible de récupérer que quelques colonne tout en conservant l'instance de Raw_data.
# Vu les soucis pour obtenir des instances de Raw_data avec un select sur un sous ensemble de colonne on prend toutes les colonnes, le soucis est peut-être lié a la mecanique de SQLAlchemy qui necessite un record complet pour retourner une instance

stm_client_groupe = select(Raw_data).group_by(  # GROUP BY ...
        Raw_data.nom,
        Raw_data.prenom,
        Raw_data.nationalite_francaise,
        Raw_data.sexe
    )

# execution de la requete

liste_client = session.scalars(stm_client_groupe).all()
# On test la requetes SQL direct
print(session.execute(sa.text("SELECT * FROM raw_data GROUP BY nom, prenom, sexe, nationalite_francaise LIMIT 10")).all())

# Populate de la table
if populate_client:
    for ligne in liste_client:
        print(vars(ligne))
        new_client = Client(**{
            'nom': ligne.nom,
            'prenom': ligne.prenom,
            'sexe': ligne.sexe,
            'nationalite_francaise': ligne.nationalite_francaise,
        })
        session.add(new_client)
    session.commit()

# On peux populate directement avec
# INSERT INTO client (nom, prenom, sexe, nationalite_francaise) SELECT nom, prenom, sexe, nationalite_francaise FROM raw_data GROUP BY nom, prenom, sexe, nationalite_francaise
# On se force à utiliser l'ORM, il apparait que cette approche est bien plus lente


SELECT raw_data.nom, raw_data.prenom, raw_data.nationalite_francaise, raw_data.sexe 
FROM raw_data GROUP BY raw_data.nom, raw_data.prenom, raw_data.nationalite_francaise, raw_data.sexe
[(272, 'Abbott', 'Adam', 74, 165.2, 57.1, 'F', 'oui', 'doctorat', 'Hauts-de-France', 'oui', 'oui', 2270.0, 'veuf', 4.0, 0.92, '2021-12-07', 576.0, 1220.4, 9981.382567383063), (1424, 'Acevedo', 'Pamela', 67, 176.1, 66.5, 'F', 'oui', 'master', 'Bretagne', 'non', 'oui', 2141.0, 'veuf', 3.0, 0.62, '2024-12-19', 581.0, 913.2, 25261.091226700035), (2678, 'Acosta', 'Douglas', 71, 166.6, 73.6, 'H', 'oui', 'doctorat', 'Corse', 'non', 'oui', 1579.0, 'divorcé', 4.0, 0.81, '2024-07-30', 576.0, 836.5600000000001, 17478.508978244045), (553, 'Acosta', 'Sarah', 59, 154.3, 80.3, 'H', 'oui', 'aucun', 'Île-de-France', 'oui', 'non', 864.0, 'veuf', 5.0, 0.76, '2021-12-01', 828.0, 270.52, 500.0), (3966, 'Adams', 'Amy', 56, 177.9, 63.9, 'H', 'oui', 'bac+2', 'Île-de-France', 'non', 'oui', 2988.0, 'divorcé', 5.0, 0.29, '2021-05-

### Remarques sur SQLAlchemy

Les requetes se font via les fonction `select()`/`delete()`/`update()`/etc... ca génère une requete utilsé par `session.execute()` ou `session.scalars()`

un print d'une des fonction `select()` et autres imprime la requetes SQL, voir si ca formate suivant le dialect SQL utilisé.

On peux executer du SQL direct en utilisant `text()`

In [13]:
# Remplissage de pret
# liste_region = df['region'].unique()
# print(Region.__tablename__)
# session.execute(delete(Region))
# session.commit()

def fill_dictionnaire(field_or_keys, DataBase: Base):
    """
    Cette fonction permet de remplir un dictionnaire
    :param field_or_keys: Tableau de valeurs ou non de champ dans le dataframe
    :param DataBase: Classe de données à compléter
    :return:
    """
    # On commence par truncate la table
    if isinstance(field_or_keys, str):
        tableau = df[field_or_keys].unique()
    else:
        tableau = field_or_keys
    session.execute(delete(DataBase))
    for ligne in tableau:
        print(ligne)
        region = DataBase(**{'nom': ligne})
        session.add(region)

    # Ce coup ci on commit apres avoir ajouter tous les objet Region
    session.commit()


if populate_dico:
    fill_dictionnaire('region', Region)
    # Pour le niveau d'étude, on met les valeurs a la main pour accorder un "score" via l'ID (on peux faire autrement mais bon... on fait short)
    fill_dictionnaire(['aucun', 'bac', 'bac+2', 'master', 'doctorat'], Niveau_etude)


# Table Pret

Maintenant on se charge de la table Pret.

Principe :
* On parcourt la table des clients
* On cherche dans raw_data les lignes appartenant au client
* On ajoute la ou les lignes de raw data dans le client, on compte sur SQLAlchemy pour faire le remplissage dans les bonnes tables.
* On en profite pour mettre à jour les dictionnaires

In [14]:
populate_pret = True
if populate_pret:
    session.execute(delete(Pret))
    session.commit()

    liste_client = session.scalars(select(Client))
    for client in liste_client:
        # let's go
        print(f"Traitement client N°{client.id} ({client.prenom} {client.nom})")
        raw_data = session.scalars(select(Raw_data).where(sa.and_(
            Raw_data.nom == client.nom,
            Raw_data.prenom == client.prenom,
            Raw_data.sexe == client.sexe,
            Raw_data.nationalite_francaise == client.nationalite_francaise,
        ))).all()
        for raw in raw_data:
            # On met les données "brute"
            ligne_pret = Pret(
                age=raw.age,
                taille=raw.taille,
                poids=raw.poids,
                sport_licence=raw.sport_licence,
                smoker=raw.smoker,
                revenu_estime_mois=raw.revenu_estime_mois,
                situation_familiale=raw.situation_familiale,
                historique_credits=raw.historique_credits,
                risque_personnel=raw.risque_personnel,
                date_creation_compte=raw.date_creation_compte,
                score_credit=raw.score_credit,
                loyer_mensuel=raw.loyer_mensuel,
                montant_pret=raw.montant_pret
            )

            # On ajoute ligne_pret à la session pour qu'elle soit connu dans la suite du traitement.
            # Ca evite le warning décrit plus bas
            session.add(ligne_pret)

            # On ajoute maintenant les liaisons
            # Region
            region = session.scalars(select(Region).where(Region.nom == raw.region)).one()
            ligne_pret.region = region
            # Niveau Etude
            niveau_etude = session.scalars(select(Niveau_etude).where(Niveau_etude.nom == raw.niveau_etude)).one()
            ligne_pret.niveau_etude = niveau_etude

            # On ajoute maintenant la ligne de pret au client.
            client.prets.append(ligne_pret)


            print(region.__dict__)
            print(raw.age, raw.taille, raw.poids)

    session.commit()

        # print(type(raw_data))
        # print(raw_data)

Traitement client N°1 (Adam Abbott)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f50fe59cad0>, 'nom': 'Hauts-de-France', 'id': 4}
74 165.2 57.1
Traitement client N°2 (Pamela Acevedo)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f50fe59df10>, 'nom': 'Bretagne', 'id': 3}
67 176.1 66.5
Traitement client N°3 (Douglas Acosta)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f50fe59e030>, 'nom': 'Corse', 'id': 8}
71 166.6 73.6
Traitement client N°4 (Sarah Acosta)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f50fe59e150>, 'nom': 'Île-de-France', 'id': 1}
59 154.3 80.3
Traitement client N°5 (Amy Adams)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f50fe59e150>, 'nom': 'Île-de-France', 'id': 1}
56 177.9 63.9
Traitement client N°6 (Brandi Adams)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f50fe59e450>, 'nom': 'Auvergne-Rhône-Alpes', 'id': 2

Voila les données sont importées

Petite remarque : j'ai des warning lors de la création des contrats

```
/tmp/ipykernel_15965/3707726440.py:38: SAWarning: Object of type <Pret> not in session, add operation along 'Region.prets' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  niveau_etude = session.scalars(select(Niveau_etude).where(Niveau_etude.nom == raw.niveau_etude)).one()
/tmp/ipykernel_15965/3707726440.py:42: SAWarning: Object of type <Pret> not in session, add operation along 'Niveau_etude.prets' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happended while initializing objects.)
  client.prets.append(ligne_pret)
```

Il faudra voir pourquoi.

In [47]:
print(select(Client).where(sa.and_(Client.prenom == client.prenom, Client.nom == client.nom)))
print(sa.text("select * from client"))

result = session.execute(select(Client).limit(10)).all()
print(result)
for row in result:
    print(row)
    for item in row:
        print(item.nom)


SELECT client.id, client.nom, client.prenom, client.sexe, client.nationalite_francaise 
FROM client 
WHERE client.prenom = :prenom_1 AND client.nom = :nom_1
select * from client
[(<__main__.Client object at 0x7f50fe954dd0>,), (<__main__.Client object at 0x7f50fe943950>,), (<__main__.Client object at 0x7f50fe940aa0>,), (<__main__.Client object at 0x7f50fe955730>,), (<__main__.Client object at 0x7f50fe956510>,), (<__main__.Client object at 0x7f50fe956240>,), (<__main__.Client object at 0x7f50fe9548f0>,), (<__main__.Client object at 0x7f50fe954ce0>,), (<__main__.Client object at 0x7f50fdc82900>,), (<__main__.Client object at 0x7f50fe967560>,)]
(<__main__.Client object at 0x7f50fe954dd0>,)
Abbott
(<__main__.Client object at 0x7f50fe943950>,)
Acevedo
(<__main__.Client object at 0x7f50fe940aa0>,)
Acosta
(<__main__.Client object at 0x7f50fe955730>,)
Acosta
(<__main__.Client object at 0x7f50fe956510>,)
Adams
(<__main__.Client object at 0x7f50fe956240>,)
Adams
(<__main__.Client object at 0x7f50